In [3]:
#Packages
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [6]:
#Importation of networks and attributes
ELadvice=  nx.read_edgelist('ELadvice_edgelist.txt',comments="node",create_using=nx.DiGraph(),nodetype=int)
ELcowork =  nx.read_edgelist('ELcowork_edgelist.txt',comments="node",create_using=nx.Graph(),nodetype=int)
ELfriend = nx.read_edgelist('ELfriend_edgelist.txt',comments="node",create_using=nx.DiGraph(),nodetype=int)
attr = pd.read_table('ELnodeAttributes.txt', sep = "\t")

In [7]:
#On associe attributs au réseau ami
ELNodeData = attr.set_index('Node').to_dict('index').items()
ELfriend.add_nodes_from(ELNodeData)

In [8]:
#Compte de la répartition genrée 
attr['Gender'].value_counts(normalize=True)

Gender
1    0.746479
2    0.253521
Name: proportion, dtype: float64

In [29]:
Deg = dict(ELfriend.in_degree())

In [30]:
genre_degrees = {}
for node, data in ELNodeData:  # itérant à travers chaque élément du dict_items
    genre = data.get('Gender')  # Récupérer le genre à partir des données du nœud
    if genre:
        if genre not in genre_degrees:
            genre_degrees[genre] = []
        genre_degrees[genre].append(Deg.get(node, 0))

# Calculer le nombre moyen de degrés par genre
average_degree_by_genre = {}
for genre, degrees in genre_degrees.items():
    average_degree_by_genre[genre] = sum(degrees) / len(degrees)

# Afficher le résultat
print("Nombre moyen de degrés par genre:")
for genre, avg_degree in average_degree_by_genre.items():
    print(f"{genre}: {avg_degree}")

Nombre moyen de degrés par genre:
1: 8.056603773584905
2: 8.222222222222221
